## Search Engine with Tools and Agents

In [56]:
# Arxiv--Research
# Tools Creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [57]:
# Used In-Built Tool of Wikipedia
api_wraper_wiki=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wraper_wiki)
wiki.name

'wikipedia'

In [58]:
api_wraper_arxiv=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wraper_arxiv)
arxiv.name

'arxiv'

In [59]:
tools=[wiki,arxiv]

In [60]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Code\\DS_ML_AI\\GenAI\\section23++LangChain\\langchain_venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [61]:
# Custom Tools Creation [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

In [62]:
loader=WebBaseLoader("https://docs.langchain.com/oss/python/langchain/overview")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorDB=FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
retriever=vectorDB.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000228B9A91D90>, search_kwargs={})

In [63]:
from langchain_classic.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "langchain-search", "Search any information about langchain")
retriever_tool

Tool(name='langchain-search', description='Search any information about langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022894BC9D00>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000228B9A91D90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000022894BCB2E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000228B9A91D90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), do

In [64]:
retriever_tool.name

'langchain-search'

In [65]:
tools=[wiki, arxiv, retriever_tool]

In [66]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Code\\DS_ML_AI\\GenAI\\section23++LangChain\\langchain_venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langchain-search', description='Search any information about langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022894BC9D00>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS 

In [67]:
# Run all the tools with Agents and LLM Models
# Tools --> LLM --> AgentExecuter

import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')
groq_api_key=os.getenv('GROQ_API_KEY')
llm=ChatGroq(model="llama-3.1-8b-instant", api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000228B9A2A2D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000228B996C9D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [68]:
# !pip install langchainhub

In [70]:
# Prompt Template
from langchain_classic import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt = hub.pull("hwchase17/react")

# print(prompt)
# print(prompt.messages)
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [71]:
# Agents
from langchain_classic.agents import create_openai_tools_agent
from langchain.agents import create_agent
# agent=create_agent(model=llm, tools=tools, system_prompt=prompt)
agent=create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [72]:
# Agent Executer
from langchain_classic.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent, tools=tools, verbouse=True)
agent_executor

AgentExecutor(verbose=False, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Ta

In [73]:
agent_executor.invoke({'input':"Tell me about Langchain"})

{'input': 'Tell me about Langchain',
 'output': "Based on the results, LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It provides a pre-built agent architecture and model integrations to help users get started quickly and seamlessly incorporate LLMs into their agents and applications.\n\nSome of the benefits of using LangChain include:\n\n1. Standard model interface: LangChain standardizes how you interact with models, allowing you to seamlessly swap providers and avoid lock-in.\n2. Easy to use, highly flexible agent: LangChain's agent abstraction is designed to be easy to get started with, but also provides enough flexibility to allow for complex context engineering.\n3. Built on top of LangGraph: LangChain's agents are built on top of LangGraph, providing durable execution, streaming, human-in-the-loop support, persistence, and more.\n4. Debug with LangSmith: LangChain provides visualization tools that trace 

In [74]:
agent_executor.invoke({'input':"Tell me something about langchain overview"})

APIError: tool call validation failed: attempted to call tool 'brave_search' which was not in request.tools

In [ ]:
agent_executor.invoke({'input':"What is Transformer in NLP?"})

APIError: tool call validation failed: attempted to call tool 'brave_search' which was not in request.tools